Exercise recommendation model 2

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import pandas as pd
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
data = pd.read_csv("/content/synthetic_progress_data.csv")

In [23]:
# Recommendation model
class ExerciseRecommender:
    def __init__(self):
        self.scaler = MinMaxScaler()
        self.difficulty_mapping = {"Easy": 1, "Medium": 2, "Hard": 3}
        self.performance_mapping = {"Good": 3, "Average": 2, "Bad": 1}

    def preprocess_data(self, data):
        data = data.copy()
        data["Difficulty Level"] = data["Difficulty Level"].map(self.difficulty_mapping)
        data["Performance Score"] = data["Overall Performance"].map(self.performance_mapping)
        data[["Total Reps", "Total Time (s)"]] = self.scaler.transform(data[["Total Reps", "Total Time (s)"]])
        return data

    def fit(self, data):
        data = data.copy()
        self.scaler.fit(data[["Total Reps", "Total Time (s)"]])
        return self

    def create_user_profile(self, data, user_id):
        user_data = data[data["User ID"] == user_id]
        user_profile = user_data[["Total Reps", "Total Time (s)", "Difficulty Level", "Performance Score"]].mean()
        return user_profile.values.reshape(1, -1)

    def recommend_exercises(self, data, user_profile, top_n=5):
        exercise_vectors = data[["Total Reps", "Total Time (s)", "Difficulty Level", "Performance Score"]].values
        similarities = cosine_similarity(user_profile, exercise_vectors).flatten()
        data["Similarity"] = similarities

        exercise_summary = (
            data.groupby("Exercise Name")
            .agg({
                "Similarity": "mean",
                "Total Reps": "mean",
                "Difficulty Level": "mean",
                "Performance Score": "mean",
                "Total Time (s)": "mean"
            })
            .reset_index()
        )

        exercise_summary["Adjusted Similarity"] = (
            exercise_summary["Similarity"] * (1 + (3 - exercise_summary["Performance Score"]) * 0.2)
        )
        recommendations = exercise_summary.sort_values(by="Adjusted Similarity", ascending=False).head(top_n)

        return recommendations[["Exercise Name"]]

In [24]:
recommender = ExerciseRecommender()
recommender.fit(data)

In [25]:
with open('recommender2.pkl', 'wb') as file:
    pickle.dump(recommender, file)

with open('scaler2.pkl', 'wb') as file:
    pickle.dump(recommender.scaler, file)

In [26]:
user_id = 1

processed_data = recommender.preprocess_data(data)
user_profile = recommender.create_user_profile(processed_data, user_id)
recommendations = recommender.recommend_exercises(processed_data, user_profile)

In [27]:
print(f"Top 5 recommendations for User 1:")
print(recommendations)

Top 5 recommendations for User 1:
       Exercise Name
5     Neck Side Bend
3    Neck Isometrics
0         Chin Tucks
7  Seated Head Turns
1     Neck Extension
